# T0. trainer 和 evaluator 的基本使用

&emsp; 1 &ensp; trainer 和 evaluator 的基本关系
 
&emsp; &emsp; 1.1 &ensp; trainer 和 evaluater 的初始化

&emsp; &emsp; 1.2 &ensp; driver 的含义与使用要求

&emsp; &emsp; 1.3 &ensp; trainer 内部初始化 evaluater

&emsp; 2 &ensp; 使用 trainer 训练模型

&emsp; &emsp; 2.1 &ensp; argmax 模型实例

&emsp; &emsp; 2.2 &ensp; trainer 的参数匹配

&emsp; &emsp; 2.3 &ensp; trainer 的实际使用 

&emsp; 3 &ensp; 使用 evaluator 评测模型
 
&emsp; &emsp; 3.1 &ensp; trainer 外部初始化的 evaluator

&emsp; &emsp; 3.2 &ensp; trainer 内部初始化的 evaluator 

## 1. trainer 和 evaluator 的基本关系

### 1.1  trainer 和 evaluator 的初始化

在`fastNLP 0.8`中，**`Trainer`模块和`Evaluator`模块分别表示“训练器”和“评测器”**

&emsp; 对应于之前的`fastNLP`版本中的`Trainer`模块和`Tester`模块，其定义方法如下所示

在`fastNLP 0.8`中，需要注意，在同个`python`脚本中先使用`Trainer`训练，然后使用`Evaluator`评测

&emsp; 非常关键的问题在于**如何正确设置二者的`driver`**。这就引入了另一个问题：什么是 `driver`？


```python
trainer = Trainer(
    model=model,
    train_dataloader=train_dataloader,
    optimizers=optimizer,
	...
	driver="torch",
	device=0,
	...
)
...
evaluator = Evaluator(
    model=model,
    dataloaders=evaluate_dataloader,
    metrics={'acc': Accuracy()} 
    ...
    driver=trainer.driver,
	device=None,
    ...
)
```

### 1.2  driver 的含义与使用要求

在`fastNLP 0.8`中，**`driver`**这一概念被用来表示**控制具体训练的各个步骤的最终执行部分**

&emsp; 例如神经网络前向、后向传播的具体执行、网络参数的优化和数据在设备间的迁移等

在`fastNLP 0.8`中，**`Trainer`和`Evaluator`都依赖于具体的`driver`来完成整体的工作流程**

&emsp; 具体`driver`与`Trainer`以及`Evaluator`之间的关系请参考`fastNLP 0.8`的框架设计

注：在同一脚本中，`Trainer`和`Evaluator`使用的`driver`应当保持一致

&emsp; 一个不能违背的原则在于：**不要将多卡的`driver`前使用单卡的`driver`**（？？？），这样使用可能会带来很多意想不到的错误。

### 1.3 Trainer 内部初始化 Evaluator

在`fastNLP 0.8`中，如果在**初始化`Trainer`时**，**传入参数`evaluator_dataloaders`和`metrics`**

&emsp; 则在`Trainer`内部，也会初始化单独的`Evaluator`来帮助训练过程中对验证集的评测

```python
trainer = Trainer(
    model=model,
    train_dataloader=train_dataloader,
    optimizers=optimizer,
	...
	driver="torch",
	device=0,
	...
    evaluate_dataloaders=evaluate_dataloader,
    metrics={'acc': Accuracy()},
	...
)
```

## 2. 使用 trainer 训练模型

### 2.1 argmax 模型实例

本节将通过训练`argmax`模型，简单介绍如何`Trainer`模块的使用方式

&emsp; 使用`pytorch`定义`argmax`模型，输入一组固定维度的向量，输出其中数值最大的数的索引

&emsp; 除了添加`pytorch`要求的`forward`方法外，还需要添加 **`train_step`** 和 **`evaluate_step`** 这两个方法

In [ ]:
import torch
import torch.nn as nn

class ArgMaxModel(nn.Module):
    def __init__(self, num_labels, feature_dimension):
        super(ArgMaxModel, self).__init__()
        self.num_labels = num_labels

        self.linear1 = nn.Linear(in_features=feature_dimension, out_features=10)
        self.ac1 = nn.ReLU()
        self.linear2 = nn.Linear(in_features=10, out_features=10)
        self.ac2 = nn.ReLU()
        self.output = nn.Linear(in_features=10, out_features=num_labels)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.ac1(self.linear1(x))
        x = self.ac2(self.linear2(x))
        x = self.output(x)
        return x

    def train_step(self, x, y):
        x = self(x)
        return {"loss": self.loss_fn(x, y)}

    def evaluate_step(self, x, y):
        x = self(x)
        x = torch.max(x, dim=-1)[1]
        return {"pred": x, "target": y}

在`fastNLP 0.8`中，**函数`train_step`是`Trainer`中参数`train_fn`的默认值**

&emsp; 由于，在`Trainer`训练时，**`Trainer`通过参数`_train_fn_`对应的模型方法获得当前数据批次的损失值**

&emsp; 因此，在`Trainer`训练时，`Trainer`首先会寻找模型是否定义了`train_step`这一方法

&emsp; &emsp; 如果没有找到，那么`Trainer`会默认使用模型的`forward`函数来进行训练的前向传播过程

注：在`fastNLP 0.8`中，`Trainer`要求模型通过`train_step`来返回一个字典，将损失值作为`loss`的键值

&emsp; 此外，这里也可以通过传入`Trainer`的参数`output_mapping`来实现高度化的定制，具体请见这一note（？？？）

同样，在`fastNLP 0.8`中，**函数`evaluate_step`是`Evaluator`中参数`evaluate_fn`的默认值**

&emsp; 在`Evaluator`测试时，**`Evaluator`通过参数`evaluate_fn`对应的模型方法获得当前数据批次的评测结果**

&emsp; 从用户角度，模型通过`evaluate_step`方法来返回一个字典，内容与传入`Evaluator`的`metrics`一致

<!-- &emsp; 从模块角度，`fastNLP 0.8`会匹配该字典的键值和一个`metric`的更新函数的函数签名，自动地将`metric`所需要的内容传给该`metric`，也就是我们会自动进行“**参数匹配**”。 -->

### 2.2 trainer 的参数匹配

`fastNLP 0.8`中的参数匹配涉及到两个方面，一是在模型训练或者评测的前向传播过程中，如果从`dataloader`中出来一个`batch`的数据是一个字典，那么我们会查看模型的`train_step`和`evaluate_step`方法的参数签名，然后对于每一个参数，我们会根据其名字从 batch 这一字典中选择出对应的数据传入进去。例如在接下来的定义`Dataset`的部分，注意`ArgMaxDatset`的`__getitem__`方法，您可以通过在`Trainer`和`Evaluator`中设置参数 `model_wo_auto_param_call`来关闭这一行为。当您关闭了这一行为后，我们会将`batch`直接传给您的`train_step`、`evaluate_step`或者 `forward`函数。

二是在传入`Trainer`或者`Evaluator metrics`后，我们会在需要评测的时间点主动调用`metrics`来对`evaluate_dataloaders`进行评测，这一功能主要就是通过对`metrics`的`update`方法和一个`batch`的数据进行参数评测实现的。首先需要明确的是一个 metric 的计算通常分为 `update` 和 `get_metric`两步，其中`update`表示更新一个`batch`的评测数据，`get_metric` 表示根据已经得到的评测数据计算出最终的评测值，例如对于 `Accuracy`来说，其在`update`的时候会更新一个`batch`计算正确的数量 right_num 和计算错误的数量 total_num，最终在 `get_metric` 时返回评测值`right_num / total_num`。

因为`fastNLP 0.8`的`metrics`是自动计算的（只需要传给`Trainer`或者`Evaluator`），因此其一定依赖于参数匹配。对于从`evaluate_dataloader`中生成的一个`batch`的数据，我们会查看传给 `Trainer`（最终是传给`Evaluator`）和`Evaluator`的每一个`metric`，然后查看其`update`函数的函数签名，然后根据每一个参数的名字从`batch`字典中选择出对应的数据传入进去。

### 2.3 trainer的实际使用

接下来我们创建用于训练的 dataset，其接受三个参数：数据维度、数据量和随机数种子，生成指定数量的维度为 `feature_dimension` 向量，而每一个向量的标签就是该向量中最大值的索引。

In [2]:
from torch.utils.data import Dataset

class ArgMaxDatset(Dataset):
    def __init__(self, feature_dimension, data_num=1000, seed=0):
        self.num_labels = feature_dimension
        self.feature_dimension = feature_dimension
        self.data_num = data_num
        self.seed = seed

        g = torch.Generator()
        g.manual_seed(1000)
        self.x = torch.randint(low=-100, high=100, size=[data_num, feature_dimension], generator=g).float()
        self.y = torch.max(self.x, dim=-1)[1]

    def __len__(self):
        return self.data_num

    def __getitem__(self, item):
        return {"x": self.x[item], "y": self.y[item]}

现在准备好数据和模型。

In [3]:
from torch.utils.data import DataLoader

train_dataset = ArgMaxDatset(feature_dimension=10, data_num=1000)
evaluate_dataset = ArgMaxDatset(feature_dimension=10, data_num=100)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
evaluate_dataloader = DataLoader(evaluate_dataset, batch_size=8)

# num_labels 设置为 10，与 feature_dimension 保持一致，因为我们是预测十个位置中哪一个的概率最大。
model = ArgMaxModel(num_labels=10, feature_dimension=10)

将优化器也定义好。

In [4]:
from torch.optim import SGD

optimizer = SGD(model.parameters(), lr=0.001)

现在万事俱备，开始使用 Trainer 进行训练！

In [5]:
from fastNLP import Trainer

# 定义一个 Trainer
trainer = Trainer(
    model=model,
    driver="torch",    # 使用 pytorch 进行训练
    device=0,          # 使用 GPU：0
    train_dataloader=train_dataloader,
    optimizers=optimizer,
    n_epochs=10,       # 训练 40 个 epoch
    progress_bar="rich"
)
dir(trainer)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_callback_called_legality',
 '_check_train_batch_loop_legality',
 '_custom_callbacks',
 '_driver',
 '_evaluate_dataloaders',
 '_fetch_matched_fn_callbacks',
 '_set_num_eval_batch_per_dl',
 '_train_batch_loop',
 '_train_dataloader',
 '_train_step',
 '_train_step_signature_fn',
 'accumulation_steps',
 'add_callback_fn',
 'backward',
 'batch_idx_in_epoch',
 'batch_step_fn',
 'callback_manager',
 'check_batch_step_fn',
 'cur_epoch_idx',
 'data_device',
 'dataloader',
 'device',
 'driver',
 'driver_name',
 'epoch_validate',
 'evaluate_batch_step_fn',
 'evaluate_dataloaders',
 'evaluate_every',
 'eval

In [8]:
import inspect 

print(inspect.getfullargspec(trainer.run))

FullArgSpec(args=['self', 'num_train_batch_per_epoch', 'num_eval_batch_per_dl', 'num_eval_sanity_batch', 'resume_from', 'resume_training', 'catch_KeyboardInterrupt'], varargs=None, varkw=None, defaults=(-1, -1, 2, None, True, None), kwonlyargs=[], kwonlydefaults=None, annotations={'num_train_batch_per_epoch': <class 'int'>, 'num_eval_batch_per_dl': <class 'int'>, 'num_eval_sanity_batch': <class 'int'>, 'resume_from': <class 'str'>, 'resume_training': <class 'bool'>})


没有问题，那么开始真正的训练！

In [9]:
trainer.run()

Output()

## 3. 使用 evaluator 评测模型

模型训练好了我们开始使用 Evaluator 进行评测，查看效果怎么样吧。

In [10]:
from fastNLP import Evaluator
from fastNLP.core.metrics import Accuracy

evaluator = Evaluator(
    model=model,
    driver=trainer.driver,  # 使用 trainer 已经启动的 driver；
    device=None,
    dataloaders=evaluate_dataloader,
    metrics={'acc': Accuracy()}  # 注意这里一定得是一个字典；
)

In [11]:
dir(evaluator)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_dist_sampler',
 '_evaluate_batch_loop',
 '_evaluate_step',
 '_evaluate_step_signature_fn',
 '_metric_wrapper',
 '_metrics',
 'dataloaders',
 'device',
 'driver',
 'evaluate_batch_loop',
 'evaluate_batch_step_fn',
 'evaluate_fn',
 'evaluate_step',
 'finally_progress_bar',
 'get_dataloader_metric',
 'input_mapping',
 'metrics',
 'metrics_wrapper',
 'model',
 'model_use_eval_mode',
 'move_data_to_device',
 'output_mapping',
 'progress_bar',
 'remove_progress_bar',
 'reset',
 'run',
 'separator',
 'start_progress_bar',
 'update',
 'update_progress_bar',
 'verbose']

In [12]:
evaluator.run()

{'acc#acc': 0.3}

{'acc#acc': 0.3}

## 4. 在 trainer 中加入 metric 来自动评测；

现在我们尝试在训练过程中进行评测。

In [13]:
# 重新定义一个 Trainer

trainer = Trainer(
    model=model,
    driver=trainer.driver,  # 因为我们是在同一脚本中，因此这里的 driver 同样需要重用；
    train_dataloader=train_dataloader,
    evaluate_dataloaders=evaluate_dataloader,
    metrics={'acc': Accuracy()},
    optimizers=optimizer,
    n_epochs=10,  # 训练 40 个 epoch；
    evaluate_every=-1,  # 表示每一个 epoch 的结束会进行 evaluate；
)

再次训练。

In [14]:
trainer.run()

In [15]:
evaluator = Evaluator(
    model=model,
    driver=trainer.driver,  # 使用 trainer 已经启动的 driver；
    dataloaders=evaluate_dataloader,
    metrics={'acc': Accuracy()}  # 注意这里一定得是一个字典；
)

In [16]:
evaluator.run()

{'acc#acc': 0.5}

{'acc#acc': 0.5}